In [1]:
%load_ext autoreload

%autoreload 2


In [2]:
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
import yfinance as yf
from tqdm import tqdm

while 'notebooks' in os.getcwd():
    os.chdir("..")
    
from src.preprocessing.get_news import get_news, \
    filter_news_with_name, save_company_news_df,\
    read_company_news_df
from src.preprocessing.sentiment import generate_sentiment_from_title

api_key = '64d11b117233d0.77790833'

In [3]:
from transformers import BertTokenizer, BertForSequenceClassification

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

2023-08-16 22:14:37.687452: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-16 22:14:37.752817: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-08-16 22:14:37.753545: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-16 22:14:38.819812: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Data acquisition

In [4]:
start_year = 2019
end_year = 2023
stock = "MSFT"

microsoft_news, microsoft_complete = get_news(stock)

100%|██████████| 12/12 [00:51<00:00,  4.27s/it]


In [5]:
len(microsoft_complete)

11431

In [6]:
news_with_microsoft = filter_news_with_name(microsoft_complete, 'Microsoft')

In [7]:
len(news_with_microsoft)

3502

In [9]:
sentiment_microsoft = generate_sentiment_from_title(
    news_with_microsoft,
    tokenizer,
    finbert
)

100%|██████████| 3502/3502 [04:26<00:00, 13.13it/s]


In [10]:
microsoft_df = pd.DataFrame(news_with_microsoft)[["date","title","content"]]

In [11]:
microsoft_df["sentiment"] = sentiment_microsoft

In [14]:
microsoft_df["date"] = pd.to_datetime(microsoft_df["date"])
microsoft_df["date"] = microsoft_df["date"].dt.date

In [15]:
save_company_news_df(microsoft_df,
                     "microsoft")